In [1]:
#Imports
import pandas as pd

In [2]:
#load data
df = pd.read_csv("BTC-USD.csv")
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,2.105680e+07
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,3.448320e+07
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,3.791970e+07
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,3.686360e+07
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,2.658010e+07
...,...,...,...,...,...,...,...
2637,2021-12-06,49413.480469,50929.519531,47281.035156,50582.625000,50582.625000,3.770731e+10
2638,2021-12-07,50581.828125,51934.781250,50175.808594,50700.085938,50700.085938,3.367681e+10
2639,2021-12-08,50667.648438,51171.375000,48765.988281,50504.796875,50504.796875,2.847970e+10
2640,2021-12-09,50450.082031,50797.164063,47358.351563,47672.121094,47672.121094,2.960358e+10


In [3]:
#Select date and price
df = df[['Date', 'Close']]
#Rename features
df = df.rename(columns={'Date':'ds', 'Close':'y'})
#Show the data
df

,ds,y
0,2014-09-17,457.334015
1,2014-09-18,424.440002
2,2014-09-19,394.795990
3,2014-09-20,408.903992
4,2014-09-21,398.821014
...,...,...
2637,2021-12-06,50582.625000
2638,2021-12-07,50700.085938
2639,2021-12-08,50504.796875
2640,2021-12-09,47672.121094


In [4]:
#Getting last 90 rows of data
last = df[len(df)-90:]
last

,ds,y
2552,2021-09-12,46063.269531
2553,2021-09-13,44963.074219
2554,2021-09-14,47092.492188
2555,2021-09-15,48176.347656
2556,2021-09-16,47783.359375
...,...,...
2637,2021-12-06,50582.625000
2638,2021-12-07,50700.085938
2639,2021-12-08,50504.796875
2640,2021-12-09,47672.121094


In [5]:
#Get all rows in dataset except last 90
df = df[:-90]
df

,ds,y
0,2014-09-17,457.334015
1,2014-09-18,424.440002
2,2014-09-19,394.795990
3,2014-09-20,408.903992
4,2014-09-21,398.821014
...,...,...
2547,2021-09-07,46811.128906
2548,2021-09-08,46091.390625
2549,2021-09-09,46391.421875
2550,2021-09-10,44883.910156


In [6]:
#Getting Facebook Prophet library  
from fbprophet import Prophet
#Creating the Prophet object
fbp = Prophet(daily_seasonality = True)
fbp.fit(df)
future = fbp.make_future_dataframe(periods=365)
forecast = fbp.predict(future)

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [7]:
#Importing facebooks prophet plotting libraries
from fbprophet.plot import plot_plotly
#plot the data
plot_plotly(fbp, forecast)

In [8]:
#Show model prediction for 12-01-2021
forecast[forecast.ds == '2021-12-01']['yhat']

2632    59333.66219
Name: yhat, dtype: float64

In [9]:
#Show acutal value of 12-01-2021
last[last.ds == '2021-12-01']['y']

2632    57229.828125
Name: y, dtype: float64